# Notebook 1: 数据探索 / Data Exploration

本笔记本教你如何获取和探索加密货币数据
This notebook teaches you how to fetch and explore cryptocurrency data

## 学习目标 / Learning Objectives
- 了解加密货币数据结构 / Understand cryptocurrency data structure
- 学习使用 Binance API / Learn to use Binance API
- 进行基本的数据探索 / Perform basic data exploration
- 可视化价格走势 / Visualize price trends

In [ ]:
# 导入必要的库 / Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# 设置中文显示 / Set up Chinese display
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 添加项目路径 / Add project path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

print("✅ 库导入成功 / Libraries imported successfully")

## 步骤 1: 从 Binance 获取数据 / Step 1: Fetch Data from Binance

In [ ]:
from utils.binance_client import BinanceUtility

# 创建 Binance 客户端 / Create Binance client
client = BinanceUtility()

# 定义交易对 / Define trading pair
symbol = 'BTCUSDT'
print(f"📡 正在获取 {symbol} 的历史数据... / Fetching historical data for {symbol}...")

# 获取过去 6 个月的 1 小时数据 / Fetch hourly data for the past 6 months
df = client.fetch_historical_data(symbol, '1h', '6 months ago UTC')

print(f"✅ 数据获取成功！/ Data fetched successfully!")
print(f"📊 数据行数 / Number of rows: {len(df)}")
print(f"📅 时间范围 / Time range: {df['timestamp'].min()} 到 {df['timestamp'].max()}")

## 步骤 2: 查看数据结构 / Step 2: Examine Data Structure

In [ ]:
# 显示前 5 行数据 / Display first 5 rows
print("📋 数据预览 / Data Preview:")
df.head()

In [ ]:
# 查看数据类型和缺失值 / Check data types and missing values
print("📊 数据信息 / Data Info:")
df.info()

In [ ]:
# 基本统计信息 / Basic statistics
print("📈 基本统计 / Basic Statistics:")
df.describe().round(2)

## 步骤 3: 理解 K 线数据 / Step 3: Understand K-line Data

In [ ]:
print("""📖 K 线数据说明 / K-line Data Explanation:

每条记录代表一个时间段（这里是 1 小时）的价格数据：
Each record represents price data for a time period (here, 1 hour):

• Open  (开盘价): 该时间段第一个交易的价格
• High  (最高价): 该时间段的最高价格
• Low   (最低价): 该时间段的最低价格
• Close (收盘价): 该时间段最后一个交易的价格
• Volume (成交量): 该时间段的交易总量
""")

## 步骤 4: 可视化价格走势 / Step 4: Visualize Price Trends

In [ ]:
# 绘制完整价格走势 / Plot full price trend
plt.figure(figsize=(15, 6))
plt.plot(df['timestamp'], df['close'], label='Close Price', linewidth=1, color='blue')
plt.title(f'{symbol} Price History (Last 6 Months)', fontsize=14, fontweight='bold')
plt.xlabel('Time / 时间', fontsize=12)
plt.ylabel('Price (USDT)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"📈 {symbol} 价格范围 / Price Range:")
print(f"   最高价 / Highest: ${df['high'].max():,.2f}")
print(f"   最低价 / Lowest: ${df['low'].min():,.2f}")
print(f"   当前价 / Current: ${df['close'].iloc[-1]:,.2f}")

## 步骤 5: 观察近期价格变化 / Step 5: Observe Recent Price Changes

In [ ]:
# 绘制最近 168 小时（7天）的价格 / Plot price for last 168 hours (7 days)
recent_days = 7
hours_per_day = 24
recent_data = df.tail(recent_days * hours_per_day)

plt.figure(figsize=(15, 6))
plt.plot(recent_data['timestamp'], recent_data['close'], 
         label=f'Close Price (Last {recent_days} Days)', 
         linewidth=2, color='green')
plt.title(f'{symbol} Price - Last {recent_days} Days', fontsize=14, fontweight='bold')
plt.xlabel('Time / 时间', fontsize=12)
plt.ylabel('Price (USDT)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 计算价格变化 / Calculate price change
price_change = df['close'].iloc[-1] - df['close'].iloc[-24*7]
percent_change = (price_change / df['close'].iloc[-24*7]) * 100

print(f"📊 过去 {recent_days} 天的价格变化 / Price change in the past {recent_days} days:")
print(f"   变化量 / Change: ${price_change:,.2f}")
print(f"   变化率 / Percentage: {percent_change:.2f}%")

## 步骤 6: 分析成交量 / Step 6: Analyze Volume

In [ ]:
# 创建子图：价格和成交量 / Create subplots: price and volume
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# 价格图 / Price plot
ax1.plot(df['timestamp'], df['close'], label='Close Price', linewidth=1, color='blue')
ax1.set_title(f'{symbol} Price and Volume Analysis', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price (USDT)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 成交量图 / Volume plot
ax2.bar(df['timestamp'], df['volume'], label='Volume', color='orange', alpha=0.6, width=0.8)
ax2.set_xlabel('Time / 时间', fontsize=12)
ax2.set_ylabel('Volume', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 成交量统计 / Volume Statistics:")
print(f"   平均成交量 / Average Volume: {df['volume'].mean():,.0f}")
print(f"   最大成交量 / Max Volume: {df['volume'].max():,.0f}")
print(f"   最小成交量 / Min Volume: {df['volume'].min():,.0f}")

## 步骤 7: 保存数据 / Step 7: Save Data

In [ ]:
# 创建 data 目录（如果不存在） / Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# 保存数据到 CSV / Save data to CSV
output_path = f'data/{symbol}_exploration.csv'
df.to_csv(output_path, index=False)

print(f"✅ 数据已保存到 / Data saved to: {output_path}")
print(f"📁 文件大小 / File size: {os.path.getsize(output_path) / 1024:.2f} KB")

## 🎓 总结 / Summary

在本笔记本中，你学会了：
In this notebook, you learned:

1. ✅ 如何使用 Binance API 获取历史数据
2. ✅ 如何查看和理解 K 线数据结构
3. ✅ 如何计算基本统计信息
4. ✅ 如何可视化价格和成交量趋势
5. ✅ 如何保存数据到本地文件

下一步：学习特征工程 / Next Step: Learn Feature Engineering (Notebook 2)